<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900"> **8:** 课程评估

**恭喜您（几乎）完成课程啦！** 

希望您喜欢这趟旅程，并获得了宝贵的技能，可以开始构建新颖且有趣的语言应用。现在，是时候把这些技能用于评估了！

在之前的部分，我们探讨了语言模型的各个方面，包括数据工作流工具、视觉语言模型（VLM）和扩散模型。评估中，我们将把所有这些概念结合起来，构建一个有趣的应用，您在使用图像生成器时可能已经理所当然地假设它的存在了。

### **设置**

在开始之前，让我们导入必要的库并初始化语言模型。

In [ ]:
import requests
from langchain_nvidia_ai_endpoints import ChatNVIDIA

## USE THIS ONE TO START OUT WITH. NOTE IT'S INTENTED USE AS A VISUAL LANGUAGE MODEL FIRST
# model_path="http://localhost:9000/v1"
## USE THIS ONE FOR GENERAL USE AS A SMALL-BUT-PURPOSE CHAT MODEL BEING RAN LOCALLY VIA NIM
model_path="http://nim:8000/v1"
# ## USE THIS ONE FOR ACCESS TO CATALOG OF RUNNING NIM MODELS IN `build.nvidia.com`
# model_path="http://llm_client:9000/v1"

model_name = requests.get(f"{model_path}/models").json().get("data", [{}])[0].get("id")
%env NVIDIA_BASE_URL=$model_path
%env NVIDIA_DEFAULT_MODE=open

if "llm_client" in model_path:
    model_name = "meta/llama-3.1-70b-instruct"

llm = ChatNVIDIA(model=model_name, base_url=model_path, max_tokens=5000, temperature=0)

<hr>
<br>

## **8.1：** 评估

在课程评估中，您将实现一个通常在图像生成 API 后面的常见功能；**合成提示词**。

在创建以文本为条件的扩散模型时，开发者通常会创建一个合成的富含关键词的数据集用于训练，以便让模型学到强大的定制先验。
- **在理想情况下，**一个图像生成器可以通过提示生成一幅任意的高质量图像，完美契合任何自然语言提示——只要提示本身足够有表现力。
- **实际中，**图像生成器通常会被松散的指令提示——有时也会进行训练——模型根据其训练数据对细节做出广泛的假设。这通常表现为“图像检索”，即仅对训练数据进行微小修改后生成的结果。

大多数提供商更希望让人们自由去提示模型，因此许多人选择使用从文本映射到文本的接口，将“常规人类提示”映射到“扩散输入提示”空间。

### **练习：** 受图像启发的生成

在课程练习中，您将实现一个潜在的模式，将合成提示词与视觉结合，以“创建受另一幅图像启发的图像”。该过程将分解为以下任务，并需要在最后合并以通过评估。

<div><img src="imgs/rad-assessment.png" width="800"/></div>

**注意：**任务 1 到 3 仅是实现任务 4 最终方案的构建模块。只有任务 4 的结果会被评分。如果您有信心，可以随意跳过任务 1 到 3。

<hr>
<br>

### **[任务 1]** 接收图像

首先，我们需要能够接收和推理图像。为此，请实现下面的 `ask_about_image` 方法：

**简化假设：**
- 让您的 LLM 预测一个问题和一个图像文件应该不会太难，但也可以直接硬编码。
- 根据您图像池的更新频率，最好结合一些批处理、缓存、分组和预处理的方式。也可以不进行这些优化。
- LangChain 连接器如 `ChatNVIDIA` 和 `ChatOpenAI` 确实提供了简化的接口，但也可以重复使用之前的请求代码。请选择您认为最简单的方式。

**提示：**
- 回忆一下 notebook 6 中的 VLM。也许可以用那个模型？它还在吗？
- 额外说明一下，一些模型如 GPT-4o 作为聊天和图像推理模型都非常强大。

In [ ]:
import requests
import base64

def ask_about_image(image_path, question="Describe the image"):
    ####################################################################
    ## < EXERCISE SCOPE

    ## TODO: Implement the method by connecting to a vision-language model

    ## EXERCISE SCOPE >
    ####################################################################

description = ask_about_image("imgs/agent-overview.png", "Describe the image")
print(description)

<hr>
<br>

### **[任务 2]** 图像创建

现在我们有了图像描述，试着根据这个响应生成一幅图像:

In [ ]:
from diffusers import DiffusionPipeline
import torch

## TODO: Consider initializing your diffusion pipeline outside of generate_images

## TODO: Implement this method
def generate_images(prompt):
    ####################################################################
    ## < EXERCISE SCOPE
    return []
    ## EXERCISE SCOPE >
    ####################################################################

images = generate_images(description)
for img in images:
    img.show()

<hr>
<br>

### **[任务 3]** 提示词合成

经过初步尝试后，您应该注意到这种描述太复杂了。也许如果我们通过一个支持 LLM 的接口将这两个不同的组件连接起来，能得到更好的结果？

抽象来讲，这个接口帮助将 VLM 输出域映射到扩散输入域，但实际上可以在任何两个规范之间完成。**简单来说，我们是想通过一个中间步骤，将 VLM 描述映射到扩散提示词。**

In [ ]:
# 解题思路
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableBranch
# from random import random

# sys_msg = (
#     "Please help the user. After every response, output '[stop]` if the conversation should end and [pass] otherwise."
# )

# prompt = ChatPromptTemplate.from_messages([("system", sys_msg), ("placeholder", "{messages}")])
# chain = prompt | llm | StrOutputParser()

# state = {"messages": []}
# agent_msg = ""

# while True:
#     try: 
#         if "[stop]" in agent_msg: break
#         else: pass
        
#         ## TODO: Update the messages appropriately
#         human_msg = input("\n[Human]:")
#         state["messages"] += [("human", human_msg)]

#         ## Initiate an agent buffer to accumulate agent response
#         agent_msg = ""
#         print("\n[Agent]: ", end="")
#         ## TODO: Stream the LLM's response directly to output and accumulate it
#         for token in chain.stream(state):
#             agent_msg += token
#             print(token, end="")

#         ## TODO: Update the messages list appropriately
#         state["messages"] += [("ai", agent_msg)]
#     except KeyboardInterrupt:
#         print("KeyboardInterrupt")
#         break

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

####################################################################
## < EXERCISE SCOPE

## TODO: Create a pipeline for synthetic prompts, optputting a string.

## EXERCISE SCOPE >
####################################################################

new_diff_prompt = ""

In [ ]:
images = generate_images(new_diff_prompt)
for img in images:
    img.show()

<hr>
<br>

### **[任务 4]** 工作流和迭代

**为了完成评估，将这些任务整合到一个工作流中，使以下过程变得一键化：**
- **从计算环境中获取一幅图像。**
- **计算图像的摘要。**
- **使用 LLM 为图像生成工作流创建四个不同的合成提示词。**
- **生成四幅您满意的独特图像。**

**注意：**
- 可以选择将其实现为标准函数或链。
- 返回一个 PIL 图像数组。您可以选择默认显示它们。
- 为了加快处理速度，建议尝试并行处理和批处理。

In [7]:
## TODO: Execute on assessment objective
import requests
import base64
import os 
from chatbot.conv_tool_caller import ConversationalToolCaller
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.pydantic_v1 import Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from chatbot.jupyter_tools import FileLister
def generate_images_from_image(image_url: str, num_images = 4):

    ####################################################################
    ## < EXERCISE SCOPE


    invoke_url = "http://localhost:9000/v1/chat/completions"
    stream = False
    with open(image_url, "rb") as f:
      image_b64 = base64.b64encode(f.read()).decode()
    headers = {
        "Authorization": "Bearer $API_KEY_REQUIRED_IF_EXECUTING_OUTSIDE_NGC",
        "Accept": "text/event-stream" if stream else "application/json"
    }
    
    payload = {
        "model": 'microsoft/phi-3.5-vision-instruct',
        "messages": [
            {'role': 'system', 'content': 'Describe the image'},
            {'role': 'user', 'content': [
                {'type': 'image_url', 'image_url': {'url': f'data:image/jpeg;base64,{image_b64}', 'detail': 'low'}}
            ]},
        ],
        "max_tokens": 512,
        "temperature": 0.20,
        "top_p": 0.70,
        "stream": stream
    }
    description = requests.post(invoke_url, headers=headers, json=payload)
    desc = description.json()["choices"][0]["message"]["content"]
    print(desc)
    ####################################################################
    ## < EXERCISE SCOPE

    ## TODO: Implement the method by connecting to a vision-language model

    ## EXERCISE SCOPE >
    ####################################################################
    ## TODO: Generate the description of the image provided in image_url
    original_description = description

    ## TODO: Generate four disjoint prompts, hopefully different, to feed into SDXL
    diffusion_prompts = []


    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", (
            "You are a helpful DLI Chatbot who can request and reason about notebooks."
            " Be as concise as necessary, but follow directions as best as you can."
            " Please help the user out by answering any of their questions and following their instructions."
        )),
        ("human", "Here is the notebook I want you to work with: {full_context}. Remembering this, start the conversation over."),
        ("ai", "Awesome! I will work with this as context and will restart the conversation."),
        ("placeholder", "{messages}")
    ])
    
    pipeline = (
        RunnablePassthrough.assign(full_context = dict(description))
        | chat_prompt 
        | llm 
        | StrOutputParser()
    )
    
    chat_state = {
        # "filenames": ["07_intro_agentics.ipynb"],
        # "messages": [("human", "Can you give me a summary of the notebook?")],
        ## Reason about the entire course at once. This will be much slower and does not scale to larger document pools. 
        "filenames": filenames, 
        "messages": [("human", 
            "Can you give me a summary of the course, making sure to mention every notebook?"
            " Do a paragraph per notebook, and finish by explaining big-picture ideas to help an"
            " instructor explain the material and understand which parts of the course to refer to when addressing questions."
        )],
    }
    
    short_summary = ""
    for chunk in pipeline.stream(chat_state):
        print(chunk, end="")
        short_summary += chunk
    
        ## TODO: Generate the resulting images
        generated_images = []
    
    
    from diffusers import DiffusionPipeline
    import torch
    desc = description.json()["choices"][0]["message"]["content"]
    ## TODO: Consider initializing your diffusion pipeline outside of generate_images
    
    ## TODO: Implement this method
    
    ####################################################################
    ## < EXERCISE SCOPE
    pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    ).to("cuda")
    images = pipe(prompt=desc).images
    ## EXERCISE SCOPE >
    ####################################################################
    
    images = generate_images(desc)

    for img in images:
        img.show()
    ## EXERCISE SCOPE >
    ####################################################################
        
    return generated_images, diffusion_prompts, original_description

results = []
results += [generate_images_from_image("imgs/agent-overview.png")]
results += [generate_images_from_image("imgs/multimodal.png")]
results += [generate_images_from_image("img-files/tree-frog.jpg")]
results += [generate_images_from_image("img-files/paint-cat.jpg")]

 The image presents a diagram illustrating the relationship between an agent and its environment. The agent is represented by a red rectangle, and it is connected to various components of its environment through solid and dashed lines. The environment is divided into three main sections: Tools, Memory, and Action. The Tools section includes items such as a Calendar, Calculator, and CodeInterpreter, among others. The Memory section is further divided into Short-term memory and Long-term memory. The Action section is connected to the Tools and Memory sections through dashed lines, indicating a less direct relationship. Additionally, there are sub-components within the Memory and Action sections, such as Reflection, Self-critics, Chain of thoughts, and Subgoal decomposition, which are connected to the main sections with solid lines. The overall layout suggests a flow of interaction between the agent and its environment, with the agent using tools and memory to perform actions and reflect 

ValueError: dictionary update sequence element #0 has length 128; 2 is required

<hr>
<br>

## **8.2：** 运行评估

要评估您的提交，请运行以下单元以保存您的结果，然后运行后面的单元查询评估运行器。

**遵循指示，确保所有步骤通过。**

In [ ]:
import os
import json
import requests
from PIL import Image
import re

def save_images_and_metadata(results, save_dir="generated_images"):
    os.makedirs(save_dir, exist_ok=True)
    
    # Collect all image paths and metadata
    all_metadata = []
    taken_idxs = [re.findall(r'\d+', filename) for filename in os.listdir(save_dir)]
    taken_idxs = [int(idx_list[0]) for idx_list in taken_idxs if idx_list]
    start_idx = max(taken_idxs, default=0) + 1  # Find the next available index

    for result in results:
        images, prompts, original_description = result
        img_paths = []

        # Save each image and store its path
        for idx, img in enumerate(images):
            img_path = os.path.join(save_dir, f"image_{start_idx + idx}.png")
            img.save(img_path, "PNG")
            img_paths.append(img_path)
        
        # Append metadata for the current batch
        all_metadata.append({
            "original_description": original_description,
            "prompts": prompts,
            "image_paths": img_paths
        })
        start_idx += len(images)
    
    # Save all metadata in a single JSON file
    metadata_path = os.path.join(save_dir, "all_metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(all_metadata, f, indent=4)
    return all_metadata

## Generate your submission
submission = save_images_and_metadata(results)

## Send the submission over to the assessment runner
response = requests.post(
    "http://docker_router:8070/run_assessment", 
    json={"submission": submission},
)

response.raise_for_status()

try: 
    print(response.json().get("result"))
    if response.json().get("messages"):
        print("MESSAGES:", "\n- ".join([""] + response.json().get("messages")))
    if response.json().get("exceptions"):
        print("EXCEPTIONS:", "\n- ".join([""] + [str(v) for v in response.json().get("exceptions")]))
except:
    print("Failed To Process Assessment Response")
    print(response.__dict__)

<br>

如果您通过了评估，请返回课程页面（如下所示），点击**"ASSESS TASK"**按钮，这将为您生成课程证书。

<img src="./imgs/assess_task.png" style="width: 800px;">


<hr>
<br>

## **8.3：** 总结

### <font color="#76b900">**恭喜您完成课程!!**</font>

在结束课程之前，我们强烈建议您下载课程材料以便后续参考，然后看看课程的**"下一步"**和**反馈**部分。**感谢您抽出时间参加课程，期待在系列课程的下一个阶段再见到您！**

<br>
<hr>